In [1]:
# Import packages
import pyspark
import dxpy
import dxdata

In [2]:
# Spark initialization (Done only once; do not rerun this cell unless you select Kernel -> Restart kernel).
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

In [3]:
# Automatically discover dispensed database name and dataset id
dispensed_database = dxpy.find_one_data_object(
    classname='database', 
    name='app*', 
    folder='/', 
    name_mode='glob', 
    describe=True)
dispensed_database_name = dispensed_database['describe']['name']

dispensed_dataset = dxpy.find_one_data_object(
    typename='Dataset', 
    name='app*.dataset', 
    folder='/', 
    name_mode='glob')
dispensed_dataset_id = dispensed_dataset['id']

In [4]:
dataset = dxdata.load_dataset(id=dispensed_dataset_id)

In [5]:
participant = dataset['participant']

In [10]:
field_name_dict = {
    'sample_names': 'eid',
    'ht_prs': 'p26244',
    't2d_prs': 'p26285'
}

In [11]:

field_names = list(field_name_dict.values())

In [12]:
df = participant.retrieve_fields(names=field_names, engine=dxdata.connect(), coding_values="replace")

In [13]:
def upload_file_to_project(filename, proj_dir):
    dxpy.upload_local_file(filename, folder=proj_dir, parents=True)
    print(f"*********{filename} uploaded!!*********")
    return

In [14]:
pandas_df = df.toPandas()

In [15]:
pandas_df = pandas_df.rename(columns={v:k for k,v in field_name_dict.items()})

In [16]:
pandas_df.head()

,sample_names,ht_prs,t2d_prs
0,2538818,0.432652,0.286012
1,5093299,NaN,NaN
2,4244964,-0.471066,0.083355
3,3351884,0.052527,1.485040
4,5632043,-1.701400,-0.137393


In [17]:
start = 0
final = len(pandas_df)
block = 0
proj_dir = f"/fields/data/prs_obesity_related_info/"
nrows = 50000

while start<final:
    end = start + nrows
    table = pandas_df.iloc[start:end, :]
    filename = f"prs_obesity_related_block{block}.csv.gz"
    table.to_csv(filename, index=False)
    upload_file_to_project(filename, proj_dir)
    start += nrows
    block += 1
    

*********prs_obesity_related_block0.csv.gz uploaded!!*********
*********prs_obesity_related_block1.csv.gz uploaded!!*********
*********prs_obesity_related_block2.csv.gz uploaded!!*********
*********prs_obesity_related_block3.csv.gz uploaded!!*********
*********prs_obesity_related_block4.csv.gz uploaded!!*********
*********prs_obesity_related_block5.csv.gz uploaded!!*********
*********prs_obesity_related_block6.csv.gz uploaded!!*********
*********prs_obesity_related_block7.csv.gz uploaded!!*********
*********prs_obesity_related_block8.csv.gz uploaded!!*********
*********prs_obesity_related_block9.csv.gz uploaded!!*********
*********prs_obesity_related_block10.csv.gz uploaded!!*********
